<a href="https://colab.research.google.com/github/miguelarbesu/AE_pathogens/blob/main/notebooks/embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install dependencies
# can take ~5-10 min
%%capture
!pip3 install -U pip > /dev/null
!pip3 install -U bio_embeddings[all] > /dev/null
!pip3 install pyyaml==5.4.1 > /dev/null
!pip3 install -U biopython > /dev/null
!pip3 install -U plotly > /dev/null

In [4]:
import numpy as np
from bio_embeddings.embed import ProtTransBertBFDEmbedder
from bio_embeddings.project import tsne_reduce
import pandas as pd
from Bio import SeqIO
import plotly.express as px

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [5]:
# donwload data from Drive and unzip
%%capture
!gdown 1tK5AzBTHrLYV1QO1b_FlvXxIUEkBMRu8
!unzip "effectors.zip"
! rm "effectors.zip"

## Batch embedding: effector collections and proteome references

In [6]:
def load_fasta(fasta_path):
    sequences = []
    uniprot_ids = []
    gene_names = []
    for record in SeqIO.parse(fasta_path, "fasta"):
        sequences.append(str(record.seq))
        uniprot_ids.append(record.id.split("|")[1])
        try:
            gene_names.append(
                [
                    x.split("=")[1]
                    for x in record.description.split(" ")
                    if x.startswith("GN=")
                ][0]
            )
        except IndexError:
            gene_names.append(record.name)
    return uniprot_ids, gene_names, sequences


def make_collection_df(uniprot_ids, gene_names, sequences, collection_name):
    collection_name_list = len(uniprot_ids) * [collection_name]
    collection_df = pd.DataFrame(
        {
            "uniprot id": uniprot_ids,
            "gene name": gene_names,
            "sequence": sequences,
            "collection": collection_name_list,
        }
    )
    return collection_df


In [7]:
# load effector collections

df = pd.DataFrame(columns=["uniprot id", "gene name", "sequence", "collection"])

EPEC_df = make_collection_df(
    *load_fasta("effectors/fasta/EPEC_effectors.fasta"), "EPEC effector"
)
EHEC_df = make_collection_df(
    *load_fasta("effectors/fasta/EHEC_effectors.fasta"), "EHEC effector"
)
CR_df = make_collection_df(
    *load_fasta("effectors/fasta/CR_effectors.fasta"), "CR effector"
)

# merge
for collection_df in [EPEC_df, EHEC_df, CR_df]:
    df = df.append(collection_df, ignore_index=True)

df.shape

(92, 4)

In [8]:
df.groupby("collection").size()

collection
CR effector      28
EHEC effector    39
EPEC effector    25
dtype: int64

We have a total of 92 effectors from 3 species. As a reference, we take now 92 random sequences from their respective proteomes, taking into account the relative composition. So, we double the dataset to 184 sequences.


In [9]:
# load reference collections and take random samples

CR_proteome_df = make_collection_df(
    *load_fasta("effectors/fasta/637910.fasta"), "Proteome"
)
CR_proteome_df = CR_proteome_df.sample(28)

EHEC_proteome_df = make_collection_df(
    *load_fasta("effectors/fasta/83334.fasta"), "Proteome"
)
EHEC_proteome_df = EHEC_proteome_df.sample(39)

EPEC_proteome_df = make_collection_df(
    *load_fasta("effectors/fasta/574521.fasta"), "Proteome"
)
EPEC_proteome_df = EPEC_proteome_df.sample(25)


for proteome_sample in [CR_proteome_df, EHEC_proteome_df, EPEC_proteome_df]:
    df = df.append(proteome_sample, ignore_index=True)

df.shape


(184, 4)

In [10]:
# Define embedder and reducer
# (down)loading the embedder takes ~2 minutes
embedder = ProtTransBertBFDEmbedder()

Some weights of the model checkpoint at /root/.cache/bio_embeddings/prottrans_bert_bfd/model_directory were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
# embedding 184 sequences takes ~30 in without a GPU!
effector_embeddings = embedder.embed_many(df["sequence"])
# embed_many returns a generator, we make it a list to load them
effector_embeddings = list(effector_embeddings)
# now pool each protein's embedding and make a final array with 
# each protein's 1024 dim embedding
effector_embeddings = np.stack(
    [embedder.reduce_per_protein(e) for e in effector_embeddings]
)
effector_embeddings.shape

(184, 1024)

In [12]:
reduced_effector_embeddings = tsne_reduce(effector_embeddings,
                                          c_components=2,
                                          init="pca")
reduced_effector_embeddings.shape


/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:699: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  FutureWarning


[t-SNE] Computing 19 nearest neighbors...
[t-SNE] Indexed 184 samples in 0.001s...
[t-SNE] Computed neighbors for 184 samples in 0.113s...
[t-SNE] Computed conditional probabilities for sample 184 / 184
[t-SNE] Mean sigma: 0.130014
[t-SNE] KL divergence after 250 iterations with early exaggeration: 116.341560
[t-SNE] KL divergence after 13250 iterations: 0.369069


(184, 3)

In [13]:
px.scatter(
    x=reduced_effector_embeddings[:, 0],
    y=reduced_effector_embeddings[:, 1],
    hover_name=df["gene name"] + " - " +df["uniprot id"],
    color=df["collection"],
    color_discrete_sequence=px.colors.qualitative.T10,
    width=600,
    height=600,
        labels={
        "x": "t-SNE component 1",
        "y": "t-SNE component 2",
        "color": "Protein type",
    }
)
